In [49]:
import numpy as np
import random
# Game of chopsticks

class GameState():

    def __init__(self, playerleft = 1, playerright = 1, oppleft = 1, oppright = 1):
        self.player = {"l": playerleft, "r": playerright}
        self.opponent = {"l": oppleft, "r": oppright}
        #self.hand_order()
        self.turn = 0 # 0 for player, 1 for opponent
    
    def __str__(self):
        printleft = self.player["l"]
        printright = self.player["r"]
        printoppleft = self.opponent["l"]
        printoppright = self.opponent["r"]
        
        
        if printleft == 0:
            printleft = "~"
        
        if printright == 0:
            printright = "~"
            
        if printoppleft == 0:
            printoppleft = "~"
            
        if printoppright == 0:
            printoppright = "~"
        
        
        return "Player: " + str(printleft) + " " + str(printright) + " Opponent: " + str(printoppleft) + " " + str(printoppright)
    
    def hand_order(self):
        if self.player["l"] > self.player["r"]:
            self.player["l"], self.player["r"] = self.player["r"], self.player["l"]
        if self.opponent["l"] > self.opponent["r"]:
            self.opponent["l"], self.opponent["r"] = self.opponent["r"], self.opponent["l"]
    
    def play_turn(self, string):
        if string[0] == "a":
            return self.play_attack_turn(string[1], string[2])
        elif string[0] == "s":
            return self.play_shift_turn(int(string[1:]))
        elif string[0] == "r":
            vmoves = self.get_valid_moves()
            move = random.choice(vmoves)
            print("Move:", move)
            return self.play_turn(move)
    
    def play_attack_turn(self, turnhand, receivehand):
        
        assert turnhand in ["l", "r"], "Invalid hand"
        assert receivehand in ["l", "r"], "Invalid hand"
        
        
        if self.turn == 0:
            if self.player[turnhand] == 0:
                print("Cannot attack with an empty hand!")
                return False

            if self.opponent[receivehand] == 0:
                print("Cannot hit an empty hand!")
                return False
            
            self.opponent[receivehand] = (self.player[turnhand] + self.opponent[receivehand]) % 5
                    
        elif self.turn == 1:
            if self.opponent[turnhand] == 0:
                print("Cannot attack with an empty hand!")
                return False

            if self.player[receivehand] == 0:
                print("Cannot hit an empty hand!")
                return False
            
            self.player[receivehand] = (self.player[receivehand] + self.opponent[turnhand]) % 5
        
        self.turn = 1 - self.turn
        #self.hand_order()
        return True
    
    def play_shift_turn(self, torightcount):
        
        
        if self.turn == 0: 
            tempr = self.player["r"] + torightcount
            templ = self.player["l"] - torightcount
            
            if (tempr < 0) or (templ < 0) or (tempr >= 5) or (templ >= 5):
                print("Invalid move!")
                return False
            else:
                self.player["r"] = tempr
                self.player["l"] = templ
            
        elif self.turn == 1:
            tempr = self.opponent["r"] + torightcount
            templ = self.opponent["l"] - torightcount
            
            if (tempr < 0) or (templ < 0) or (tempr >= 5) or (templ >= 5):
                print("Invalid move!")
                return False
            else:
                self.opponent["r"] = tempr
                self.opponent["l"] = templ
        
        self.turn = 1 - self.turn
        #self.hand_order()
        return True
    def is_game_over(self):
        if self.player["l"] == 0 and self.player["r"] == 0:
            return 1
        elif self.opponent["l"] == 0 and self.opponent["r"] == 0:
            return -1
        else:
            return 0
    
    def get_valid_moves(self):
        
        vmoves = []

        if self.turn == 0:
            
            if self.player["l"] != 0:
                if self.opponent["l"] != 0:
                    vmoves.append("all")
                if self.opponent["r"] != 0:
                    vmoves.append("alr")
                
                for i in np.arange(1, self.player["l"] + 1):
                    vmoves.append("s" + str(i))
                
            if self.player["r"] != 0:
                if self.opponent["l"] != 0:
                    vmoves.append("arl")
                if self.opponent["r"] != 0:
                    vmoves.append("arr")
    
                for i in range(1, self.player["r"] + 1):
                    vmoves.append("s" + str(-i))
            
            d = self.player["r"] - self.player["l"]
            if d != 0:
                vmoves.remove("s" + str(-d))
            
        elif self.turn == 1:
            if self.opponent["l"] != 0:
                if self.player["l"] != 0:
                    vmoves.append("all")
                if self.player["r"] != 0:
                    vmoves.append("alr")
                
                for i in np.arange(1, self.opponent["l"] + 1):
                    vmoves.append("s" + str(i))
                
            if self.opponent["r"] != 0:
                if self.player["l"] != 0:
                    vmoves.append("arl")
                if self.player["r"] != 0:
                    vmoves.append("arr")
    
                for i in range(1, self.opponent["r"] + 1):
                    vmoves.append("s" + str(-i))

            d = self.opponent["r"] - self.opponent["l"]
            if d != 0:
                vmoves.remove("s" + str(-d))
            
        return vmoves
        
        

In [24]:
gs = GameState(3, 2, 0, 4)
#gs.hand_order()

In [25]:
print(gs)

Player: 3 2 Opponent: ~ 4


In [26]:
gs.get_valid_moves()

['alr', 's1', 's2', 's3', 'arr', 's-1', 's-2']

In [27]:
gs.play_turn(gs.get_valid_moves()[0])

True

In [46]:
def ask_turn(gs):
    print("Enter your move: ", end = '')
    res = input()
    print(res)
    if res == "p":
        print(gs)
        return ask_turn(gs)
    if res == "q":
        return False
    
    vmoves = gs.get_valid_moves()
    if res == "g":
        print("Valid Moves:", vmoves)
        return ask_turn(gs)
    if res not in vmoves:
        print("Invalid move!")
        return ask_turn(gs)
    return res

In [50]:
# MAIN
gs = GameState(1, 0, 0, 1)
while gs.is_game_over() == 0:
    print("*" * 20)
    
    print("Player 1 Turn")
    print(gs)
    
    res = ask_turn(gs)
    if res:
        suc = gs.play_turn(res)
        assert suc
    else:
        print("Quitting.")
        break
    
    
    if gs.is_game_over():
        print("*" * 20)
        print(gs)
        print("Game Over!")
        break
    
    print("*" * 20)
    
    print("Player 2 Turn")
    print(gs)
    res = "r" # input("Player 2 Turn")
    suc = gs.play_turn(res)
    
    if gs.is_game_over():
        print("*" * 20)
        print(gs)
        print("Game Over!")
        break
        


********************
Player 1 Turn
Player: 1 ~ Opponent: ~ 1
Enter your move: s1
Invalid move!
Enter your move: s2
Invalid move!
Enter your move: a
Invalid move!
Enter your move: g
Valid Moves: ['alr']
Enter your move: alr
********************
Player 2 Turn
Player: 1 ~ Opponent: ~ 2
Move: s-1
********************
Player 1 Turn
Player: 1 ~ Opponent: 1 1
Enter your move: alr
********************
Player 2 Turn
Player: 1 ~ Opponent: 1 2
Move: arl
********************
Player 1 Turn
Player: 3 ~ Opponent: 1 2
Enter your move: alr
********************
Player 2 Turn
Player: 3 ~ Opponent: 1 ~
Move: all
********************
Player 1 Turn
Player: 4 ~ Opponent: 1 ~
Enter your move: all
Player: 4 ~ Opponent: ~ ~
Game Over!


In [12]:
gs = GameState(3, 0, 0, 1)

In [36]:
a = [1, 2, 3]
a.remove(1)
a

[2, 3]

In [14]:
gs.get_valid_moves()

['alr', 's-1']

In [15]:
gs = GameState(3, 0, 1, 0)

In [16]:
gs.turn = 1

In [17]:
gs.get_valid_moves()

['all', 's1']

In [23]:
"s" + str(gs.player["r"] - gs.player["l"])

's0'